# Combining the Datasets
#### Combines the datasets from all .csv files into two separate DataFrames; one for trips and one for stations.

In [1]:
import pandas as pd
import os
import glob

In [2]:
#create two lists to organize this data
divvy_trip_li = []
divvy_station_li = []

#define filepath where data is stored
path = 'C:\\Users\\JackMann\\Box Sync\\IBM\\CBDS Dev-UP\\data'

#get all .csv files
all_files = glob.glob(path + "/*.csv")

#for each file
for filename in all_files:
    print("Processing ",filename)
    #if it is a trip, add it to the divvy_trip list
    if("Trips" in filename):
        df = pd.read_csv(filename, index_col=None, header=0)
        divvy_trip_li.append(df)
    #if it is a station, add it to the divvy_station list
    elif("Stations" in filename):
        df = pd.read_csv(filename, index_col=None, header=0)
        divvy_station_li.append(df)

Processing  C:\Users\JackMann\Box Sync\IBM\CBDS Dev-UP\data\Divvy_Stations_2017_Q1Q2.csv
Processing  C:\Users\JackMann\Box Sync\IBM\CBDS Dev-UP\data\Divvy_Stations_2017_Q3Q4.csv
Processing  C:\Users\JackMann\Box Sync\IBM\CBDS Dev-UP\data\Divvy_Trips_2017_Q1.csv
Processing  C:\Users\JackMann\Box Sync\IBM\CBDS Dev-UP\data\Divvy_Trips_2017_Q2.csv
Processing  C:\Users\JackMann\Box Sync\IBM\CBDS Dev-UP\data\Divvy_Trips_2017_Q3.csv
Processing  C:\Users\JackMann\Box Sync\IBM\CBDS Dev-UP\data\Divvy_Trips_2017_Q4.csv


In [4]:
#concatenate all values from the list for stations and trips into respective dataframes
divvy_trip_frame = pd.concat(divvy_trip_li, axis=0, ignore_index=True,sort=False)
divvy_station_frame = pd.concat(divvy_station_li, axis=0, ignore_index=True,sort=False)

#merge dataframes to get data for both from_station and to_station (inner joins on station_ids)
merged_df = pd.merge(divvy_trip_frame, divvy_station_frame, how='left', left_on='from_station_id',right_on='id').merge(divvy_station_frame, how='left',left_on = 'to_station_id',right_on='id')

In [5]:
#drop duplicates
merged_df.drop_duplicates(subset='trip_id', keep='first', inplace=True)

# Data Manipulation -> Age, distance, and speed

In [6]:
#get age from 2017 (time the data was collected)
merged_df['Age'] = 2017 - merged_df['birthyear']
#END AGE

In [38]:
#get distance from stations
#import distance formula
from haversine import haversine
import numpy as np
#calculate haversine distance for each row unless the row does not have latitude/longitude data

for index, row in merged_df.iterrows():
    if row['latitude_x'] is not None and row['latitude_y'] is not None and row['longitude_x'] is not None and row['longitude_y'] is not None:
        distance = haversine((row['longitude_x'], row['latitude_x']),(row['longitude_y'],row['latitude_y']),unit='mi')
        merged_df.set_value(index, 'Distance', distance)
    else:
        print("no distance calculated")

print(merged_df['Distance'])

C:\Users\JackMann\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


0           0.854513
4           0.656664
8           0.388786
12          0.792088
16          0.360549
20          0.293797
24          0.035314
28          0.234387
32          0.687332
36          0.669884
40          0.669884
44          1.209227
48          0.656664
52          0.449133
56          1.112724
60          0.035681
64          0.523344
68          0.317714
72          0.708823
76          0.476377
80          0.031968
84          0.436878
88          2.004406
92          0.984058
96          1.924784
100         0.010853
104         0.270957
108         1.820634
112         1.827570
116         1.269285
              ...   
15306076    1.562676
15306080    0.948147
15306084    0.758584
15306088    0.480280
15306092    1.593244
15306096    1.593244
15306100    0.985623
15306104    0.985623
15306108    0.000000
15306112    0.000000
15306116    0.745137
15306120    0.395324
15306124    0.395324
15306128    0.243067
15306132    1.374975
15306136    1.008266
15306140    1

In [46]:
#get speed

#speed in miles per hour
merged_df['Median Speed'] = merged_df['Distance'] / (merged_df['tripduration'])*60*60



print(merged_df['Median Speed'])

0            3.589554
4           10.016907
8            4.021926
12           9.901104
16           3.127653
20           4.370541
24           0.352158
28           2.383597
32           8.275565
36           2.055908
40           2.043712
44           8.637333
48          10.059533
52           6.315933
56           6.789505
60           0.856338
64           4.214854
68           3.476502
72           9.212143
76           1.942193
80           0.103123
84          12.287196
88           4.418776
92           5.741669
96           9.022426
100          0.285200
104          2.911778
108         10.241066
112         10.715397
116          4.266503
              ...    
15306076     7.500844
15306080     9.200348
15306084     5.677552
15306088     3.464947
15306092     4.991887
15306096     4.893923
15306100    10.591774
15306104    10.137841
15306108     0.000000
15306112     0.000000
15306116     2.024524
15306120     4.273776
15306124     1.563920
15306128     3.923953
15306132  

In [47]:
#to .csv
merged_df.to_csv('merged_divvy.csv')